# Getting Started with JobShopLab 


The **JobShopLab** is a Framework for solving real world JobShopScheduling scenarios using Reinforement Learning.


This Getting Started Guide provides some Examples and Explanations on how to use the Framework


## Table of Contents

- [Example of solving a ft06 instance with a random policy](#example)
- [Design Choices](#design_choices)
- [Configuring the Framework](#configuring-the-framework)
    - [Config File](#config-file)
    - [Dependency Injection](#dependency-injection)
- [Customizing](#customing)
- [Defining a Problem](#defining-a-problem)
    - [Spec Files](#spec-files)
    - [The Compiler](#the-compiler)
    - [DSL](#dsl)
    - [DSL as a string](#dsl-as-a-string)
- [Visualization](#visualization)
- [Defining Agents and Solving the Environment](#defining-agents-and-solving-the-environment)



In [2]:
from jupyter_utils import change_to_jobshoplab, show_mermaid
change_to_jobshoplab()

Already in the desired directory: c:\Users\hojo724\Documents\_git\_proto_lab\_jobshoplab_github\jobshoplab


**Example** of solving a ft06 instance with a random policy <a class="anchor" id="example"></a>

In [3]:
# Solve a academic ft06 instance with a random policy

from jobshoplab import JobShopLabEnv, load_config
from pathlib import Path

config = load_config(config_path = Path("./data/config/getting_started_config.yaml"))
env = JobShopLabEnv(config=config)
done = False
while not done:
    action = env.action_space.sample()
    obs, reward, truncated,terminated, info = env.step(action)
    done = truncated or terminated
env.render()

2025-03-05 14:21:55,999 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F200C8BC0> - WARNING - No binary_job_action_factory defined in config
Dash app running on http://127.0.0.1:8050/


<div id='design_choices'/>
    
# 🔥 Design Choices 

![Design Choices](../docs/FrameworkOverview.svg)


> JobShopLab is designed to be fully **extensible** and **customizable**.

At its core sits a **state machine**, which is implemented in a **functional programming style**.  
The state machine takes inputs and delivers outputs via an **immutable DataTypeObject (Interface).**  

To **address** the need for different **time mechanisms**, observation spaces, action spaces, etc.,  
there is a **software layer** called **Middleware**. The Middleware sits between the **Gym environment** and the **state machine**,  
"translating" the Gym interface to the state machine interface.  

**Observation, reward, and action factories** are injected into the Middleware to make **customization** possible.  
The interfaces to these factories are represented by **data type objects (Dataclasses).**  

### Rendering  

Rendering is done via a **dashboard with Dash**.  
The dashboard shows a **central Gantt chart** and a **table with all schedules**.  
Dashboards can be shown **inline in a Jupyter Notebook** or **via the browser**.

There are rendering utilities for debugging also.

A 3D Simulation webapp can be accessed via the browser also 

### Configuration Management  

A critical part of keeping a framework like **JobShopLab** maintainable and results reproducible  
is a **proper configuration management** system.  

There are **two types of configurations** to consider:  

#### 1. Framework Config  
A **YAML file** containing config parameters to **control** the behavior of the framework.  

Examples:  
- Setting the **observation space**  
- Setting **render modes**  
- Controlling **truncation behavior**  

#### 2. Problem Instance Config  
A **DSL file** (Domain Specific Language in YAML syntax) defining the **problem to solve**.  

Examples:  
- **Setting machine times** and **operation sequences**  
- **Defining buffer settings**  
- **Setting setup times** for machines  
- Etc.


## ⚙️ Configuring the Framework

The framework can be configured in two ways:

- Via a [config.yaml](#config-file) file
- Via [dependency injection](#dependency-injection)

A config file is always required, but it can be **overwritten** via dependency injection.  
In order to quickly test a new component, dependency injection can be quite handy.  
However, the usage of **config objects** is the recommended way of configuring the framework.


## 📃 Config File

Framework Configs are YAML syntax.
The YAML gets parsed dynamically into a Python dataclass object.
This allows:
 - Dot notation attribute access
 - Autocompletion
 - Type safety and validation

<details>
<summary>Example configuration file for JobShopLab</summary>

```yaml
# Example configuration file for JobShopLab

title: "Example Environment"
default_loglevel: "warning" # JobShopLab uses the logging module with a Logger obj for Verbosity. Set a default loglevel here


# Define all Dependencies for the environment here.. want to use a different observation? Simply change the ActionFactory name
env: 
    loglevel: "warning"
    observation_factory: "BinaryActionObservationFactory"
    reward_factory: "BinaryActionJsspReward"
    interpreter: "BinaryJobActionInterpreter"
    render_backend: "render_in_dashboard"
    middleware: "EventBasedBinaryActionMiddleware"
    max_time_fct: 2
    max_action_fct: 3

# For every Software Component there is a designated field 
compiler:
    loglevel: "warning"
    repo: "SpecRepository" # set to "YamlRepository" in order to use custom Problem Instances
    validator: "DummyValidator"
    manipulators: # want to randomize machine times? Set one or multiple Manipulators 
        - "DummyManipulator"
    yaml_repository:
        dir: "data/config/instance_proto_lab.yaml" # yaml instance file to read
    spec_repository:
        dir: "data/jssp_instances/ft06" # academic spec file to read
state_machine:
    loglevel: "warning"

middleware:
    event_based_binary_action_middleware:
        loglevel: "warning"
        truncation_joker: 5 # 
        truncation_active: False

interpreter:
    binary_job_action_interpreter:
        loglevel: "warning"
    # want to add settings for a new action_interpreter (change the action space)
    # set a new field here with name of the class as lowercase 

observation_factory:
    binary_action_observation_factory:
        loglevel: "warning"
    
    

reward_factory:
    binary_action_jssp_reward:
        loglevel: "warning"
        sparse_bias: 1 # settings are passed as kwargs to the constructor of the Class
        dense_bias: 0.001
        truncation_bias: -1

render_backend:
    render_in_dashboard:
        loglevel: "warning"
        port: 8050
        debug: False
    simulation:
        json_dump_dir: "data/tmp/simulation_interface.json"
        port: 8051
        loglevel: "warning"
        bind_all: False
```

In [4]:
# load a config file
config = load_config(config_path=Path("./data/config/getting_started_config.yaml"))

# config is a object (immutable dataclass) atributtes can be accessed with dot notation
# a stub file (.pyi) is automaticly created when the config is loaded to ensure autocompletion
print(f"Dashboard Port: {config.render_backend.render_in_dashboard.port}") 

# config is immutable to prevent accidental changes and enforce consistency
try:
    config.render_backend.render_in_dashboard.port = 1000
except AttributeError as e:
    print(f"Error: {e}")

# create an environment with the loaded config
env = JobShopLabEnv(config=config)

Dashboard Port: 8050
Error: cannot assign to field 'port'
2025-03-05 14:21:58,223 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F204CD2E0> - WARNING - No binary_job_action_factory defined in config


## ⏬ Dependency Injection

The environment allows passing dependencies directly as constructor arguments.  
The passed instances get constructed inside the environment.  
Additional arguments, which are not included in the `config.yaml`, can be passed via partial application.

This allows:
 - dynamic instance creation (useful for Hyperparameter Optimizations)
 - quick implementation of experiments
 - customizability

In [5]:
from jobshoplab.env.factories.observations import DummyObservationFactory
from functools import partial
config = load_config(config_path=Path("./data/config/getting_started_config.yaml"))

observation_factory = DummyObservationFactory # instead of using the observation factory from the config, 
# we will use a dummy observation factory that returns a random observation

observation_factory = partial(DummyObservationFactory, test_var="test_var") # we can pass additional arguments to the observation factory using partial application

env = JobShopLabEnv(config=config,observation_factory=observation_factory)
assert isinstance(env.state_simulator.observation_factory, DummyObservationFactory) # check that the observation factory is the one we passed
assert env.state_simulator.observation_factory.test_var == "test_var" # check that the argument was passed correctly

2025-03-05 14:21:58,341 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F203686E0> - WARNING - No binary_job_action_factory defined in config


## ✍️ Customing

Want to introduce a new Reward System?<br>
The Framework provides BaseClasses for every factory, allowing easy customisations<br>

> Same as for the Reward System applies to the ObservationFactory and the ActionInterpreter.<br>
AbstractBaseClasses are beeing used to provide a interface

In [6]:
from jobshoplab.env.factories.rewards import RewardFactory
from jobshoplab.types import StateMachineResult


class CustomRewardFactory(RewardFactory): # inherit from the abstract base class for the desired factory
    # note: every ActionFactory takes 3 default arguments: loglevel, config and instance, additional arguments can be passed to the constructor
    def __init__(
        self, loglevel, config, instance, bias_a, bias_b, *args, **kwargs
    ):  # add args and kwargs to the constructor as placeholders
        self.test_var = super().__init__(
            loglevel, config, instance
        )  # call the parent constructor and pass the loglevel, config and instance
        self.bias_a, self.bias_b = bias_a, bias_b

    # the make method is called to create a reward (see the abstract base class RewardFactory)
    def make(self, state_result: StateMachineResult, terminated: bool, truncated: bool) -> float:
        if not terminated or truncated:
            return self.bias_a
        else:  # if done return a reward proportional to the time taken (makespan)
            return self.bias_b * state_result.state.time.time

    def __repr__(self) -> str:  # add a custom representation to the factory
        return f" CustomRewardFactory with bias_a: {self.bias_a}, bias_b: {self.bias_b}"



In [7]:

bias_a, bias_b = 0, 1
reward_factory = partial(
    CustomRewardFactory, bias_a=bias_a, bias_b=bias_b
)  # pass additional arguments to the constructor using partial application
env = JobShopLabEnv(config=config, reward_factory=reward_factory)
assert env.reward_factory.bias_a == bias_a  # argument was passed correctly and is accessible

## lets run a random env and track the rewards
done = False
rewards = []
while not done:
    action = env.action_space.sample()
    obs, reward, truncated, terminated, info = env.step(action)
    done = truncated or terminated
    rewards.append(reward)

print(f"Rewards: {rewards}")

2025-03-05 14:21:58,371 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F1FFB3C20> - WARNING - No binary_job_action_factory defined in config
Rewards: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 90]


# ❓ Defining a Problem

In [8]:
# how_mermaid("./docs/assets/instance.mmd")



There are 3 ways of setting the Instance (Problem to solve)

1. **Spec File:** Common Academic JSSP Problem definitions found in Literature 
2. **DSL (Domain Specific Language File):** YAML file which defines the scheduling problem in a flexible way for real-world scenarios
3. **DSL string** (useful in Jupyter or for testing/debugging purposes)


### 🏫 Spec Files

Spec files are academic JSSP Problem instances like they are commonly used in literature.
Common instances are included in JobShopLab.
In order to use those.
Specify the SpecRepository in the config file:
```yaml
compiler:
  loglevel: *default_loglevel
  repo: "SpecRepository" # use the SpecRepository here
  validator: "DummyValidator"
  manipulators:
    - "DummyManipulator"
  yaml_repository:
    dir: "data/config/dsl.yaml"
  spec_repository:
    dir: "data/jssp_instances/ft06" # set the dir to the file here
```

> Alternatively use **dependency injection:**

In [9]:
from jobshoplab.compiler.repos import SpecRepository
from jobshoplab.compiler import Compiler

repo = SpecRepository(dir=Path("data/jssp_instances/ft06"),loglevel="warning",config=config)
compiler = Compiler(config=config,loglevel="warning",repo=repo)
# pass the compiler to the environment
env = JobShopLabEnv(config=config, compiler=compiler)

2025-03-05 14:21:58,481 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F204F15B0> - WARNING - No binary_job_action_factory defined in config


### 🔄 The Compiler
The Compiler generates 2 Interfaces which are used throughout the rest of JobShopLab:

* **Instance** a DataTypeObject holding all information about the Problem 
* **(init) State** a DataTypeObject representing the current State of the Schedule. The State is Time dependent, hence every state has an attribute Time

So the compiler takes input from various sources (extendable via dependency injection) and compiles a generic type interface.

In [10]:
from jobshoplab.types import InstanceConfig, State
# Compiler has a compile method that returns the compiled instance and the initial state
instance , init_state = compiler.compile()

instance : InstanceConfig
init_state : State

# Some examples of accsessing artibs 


# the instance is a dataclass object witch is a representation for the Jssp Problem Interface.
# The instance is static and does not change
# note: the instance is immutable to prevent accidental changes and enforce consistency
print("Instance")
print("Some Machine ID:", instance.machines[0].id)
print("Operation Duration:", instance.instance.specification[0].operations[0].duration)

# the initial state is a dataclass object representing the initial state of all components
# the state is dynamic meaning that it changes as the environment is stepped its basicly a snapshot of the environment
# the state gets updated by the state simulator (state_machine)
# note: the state is immutable to prevent accidental changes and enforce consistency
print("\nState:")
print("Time:", init_state.time)
print("MachineState:",init_state.machines[0].state)

Instance
Some Machine ID: m-0
Operation Duration: DeterministicDurationConfig(duration=1)

State:
Time: Time(time=0)
MachineState: MachineStateState.IDLE


### 📄 DSL

The main purpose of JobShopLab is to represent real-world scheduling problems.<br>
So there needs to be a way of specifying the Problem itself.
This is what the DSL (instance.yaml) is for.

**Specify the DslRepository in the config file:**
```yaml
compiler:
  loglevel: *default_loglevel
  repo: "DslRepository" # <- set the repo here
  validator: "DummyValidator"
  manipulators:
    - "DummyManipulator"
  dsl_repository:
    dir: "data/config/dsl.yaml" # <- set the filepath here
  spec_repository:
    dir: "data/jssp_instances/ft06"
```

>  **Alternatively** use dependency injection:

In [11]:
from jobshoplab.compiler.repos import DslRepository
from jobshoplab.compiler import Compiler

repo = DslRepository(dir=Path("data/config/getting_started_instance.yaml"),loglevel="warning",config=config)
compiler = Compiler(config=config,loglevel="warning",repo=repo)
# pass the compiler to the environment
env = JobShopLabEnv(config=config, compiler=compiler)

2025-03-05 14:21:58,522 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F20357200> - WARNING - No binary_job_action_factory defined in config


### 🔠DSL as a string

An instance can also be defined **inline as a string**.<br>
Comes in handy when working in Jupyter notebooks, for debugging and testing purposes

In [12]:
dsl_str = """
title: InstanceConfig

# Example of a 6x6 Instance
# with AGVs

instance_config:
  description: "ft06 with AGVs" 
  instance:
    description: "6x6"
    specification: |
      (m0,t)|(m1,t)|(m2,t)|(m3,t)|(m4,t)|(m5,t)
      j0|(2,1) (0,3) (1,6) (3,7) (5,3) (4,6)
      j1|(1,8) (2,5) (4,10) (5,10) (0,10) (3,4)
      j2|(2,5) (3,4) (5,8) (0,9) (1,1) (4,7)
      j3|(1,5) (0,5) (2,5) (3,3) (4,8) (5,9)
      j4|(2,9) (1,3) (4,5) (5,4) (0,3) (3,1)
      j5|(1,3) (3,3) (5,9) (0,10) (4,4) (2,1)

    transport:
      type: "agv"
      amount: 6
  logistics: 
    specification: |
      m-0|m-1|m-2|m-3|m-4|m-5|in-buf|out-buf
      m-0|0 21 16 9 37 41 19 19
      m-1|21 0 13 15 17 23 8 8
      m-2|16 13 0 13 23 28 7 7
      m-3|9 15 13 0 31 35 14 14
      m-4|37 17 23 31 0 7 25 25
      m-5|41 23 28 35 7 0 24 24
      in-buf|19 8 7 14 25 24 0 0
      out-buf|19 8 7 14 25 24 0 0
      
init_state:
  transport:
    - location: "m-0"
    - location: "m-1"
    - location: "m-2"
    - location: "m-3"
    - location: "m-4"
    - location: "m-5"
""" 

In [13]:
from jobshoplab.compiler.repos import DslStrRepository
from jobshoplab.compiler import Compiler

repo = DslStrRepository(dsl_str=dsl_str,loglevel="warning",config=config) # using the DslStrRepository here and passing the sting as an Argument
compiler = Compiler(config=config,loglevel="warning",repo=repo)
# pass the compiler to the environment
env = JobShopLabEnv(config=config, compiler=compiler)

2025-03-05 14:21:58,550 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F206589E0> - WARNING - No binary_job_action_factory defined in config



> A **full example** and explanation of the DSL can be found in "/data/examples/full_instance.yaml"

# 🖌️Visualization

In [14]:
show_mermaid("./docs/assets/rendering.mmd")

JobShopLab provides 3 main ways of visualizing an env state.
1. Gantt Chart Dash WebApp for visualizing the schedules in a Timeline
2. CLI table for debugging using the rich lib
3. SimulationWebApp using Three.js to render 3D scenes of the Schedules (Coming Soon ;))

The default render mode can be set in the config.yaml.
```yaml

env:
  loglevel: *default_loglevel
  observation_factory: "BinaryActionObservationFactory"
  reward_factory: "BinaryActionJsspReward"
  interpreter: "BinaryJobActionInterpreter"
  render_backend: "render_in_dashboard" # <- set the render backend here
  middleware: "EventBasedBinaryActionMiddleware"

# Available Options and Settings
render_backend:
  render_in_dashboard:
    loglevel: *default_loglevel
    port: 8050
    debug: false

  simulation:
    json_dump_dir: "data/tmp/simulation_interface.json"
    port: 8051
    loglevel: *default_loglevel
    bind_all: false
  
  cli_table:
    loglevel: *default_loglevel

```

When calling ***env.render()*** there can be passed a mode flag to select the render backend also

Available Flags:
- **normal**(default) uses the default backend from the config
- **dashboard** Dash Gantt chart
- **simulation** 3D three.js simulation
- **debug** rich CLI table  

In [15]:
repo = DslStrRepository(dsl_str=dsl_str,loglevel="warning",config=config) # using the DslStrRepository here
compiler = Compiler(config=config,loglevel="warning",repo=repo)
# pass the compiler to the environment
env = JobShopLabEnv(config=config, compiler=compiler)

# run with random actions
done = False
while not done:
    action = env.action_space.sample()
    obs, reward, truncated, terminated, info = env.step(action)
    done = truncated or terminated
env.render()
# env.render(mode="simulation")
# env.render(mode="debug")

2025-03-05 14:21:58,773 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F203C9580> - WARNING - No binary_job_action_factory defined in config
Dash app running on http://127.0.0.1:8051/


# 🤖 Defining Agents and Solving the Environemnt

checkout the jobshopagent repo for more information on how to define agents and solve the environment, using reinforcement learning algorithms

In [16]:
# train an agent using stable baselines3

from stable_baselines3 import PPO
from jobshoplab import JobShopLabEnv

env = JobShopLabEnv(config=config)
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=10)

2025-03-05 14:22:01,666 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F1F5E6030> - WARNING - No binary_job_action_factory defined in config
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
2025-03-05 14:22:03,349 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F282A8F50> - WARNING - No binary_job_action_factory defined in config
2025-03-05 14:22:03,506 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F283B7E30> - WARNING - No binary_job_action_factory defined in config
2025-03-05 14:22:03,663 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F28362270> - WARNING - No binary_job_action_factory defined in config
2025-03-05 14:22:03,790 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F283C7F80> - WARNING - No binary_job_action_factory defined in config
2025-03-05 14:22:03,969 - <jobshoplab.env.env.DependencyBuilder object at 0x0000021F2840E5D0> - WARNING - No binary_job_action_factory define